In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_part = {}
list_job = []
for i in range(len(input_)):
    vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
    dict_part[input_.loc[i, 'Url']] = {'vehicle_page': vehicle_page,
                                       'crawler': 0,
                                       'dict_vehicle': {},
                                       'vehicle': ''}
    for j in range(vehicle_page):
        list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                resp = requests.get(input_.loc[i, 'Url'] + '?page=' + str(j+1),
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                list_h3 = html.xpath('//section[@id="applications"]/h3[@class="part__titre-groupe-vehicules"]')
                
                # = = = = = = = = = = = = = = =
                
                list_table = html.xpath('//section[@id="applications"]/table[@class="part__table-applications"]')

                # = = = = = = = = = = = = = = =
                
                for h3, table in zip(list_h3, list_table):
                    make_model_engine = h3.text.strip()
                    list_year = sorted(list(set([int(td.text.strip()) for td in table.xpath('./tbody/tr/td[2]')])))

                    if make_model_engine not in dict_part[input_.loc[i, 'Url']]['dict_vehicle']:
                        dict_part[input_.loc[i, 'Url']]['dict_vehicle'][make_model_engine] = list_year
                    else:
                        dict_part[input_.loc[i, 'Url']]['dict_vehicle'][make_model_engine] += list_year
                        dict_part[input_.loc[i, 'Url']]['dict_vehicle'][make_model_engine] = sorted(list(set(dict_part[input_.loc[i, 'Url']]['dict_vehicle'][make_model_engine])))

                dict_part[input_.loc[i, 'Url']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_part[input_.loc[i, 'Url']]['vehicle_page'] == dict_part[input_.loc[i, 'Url']]['crawler']:
                    list_vehicle = []
                    for make_model_engine in dict_part[input_.loc[i, 'Url']]['dict_vehicle']:
                        year_min = min(dict_part[input_.loc[i, 'Url']]['dict_vehicle'][make_model_engine])
                        year_max = max(dict_part[input_.loc[i, 'Url']]['dict_vehicle'][make_model_engine])

                        if year_min == year_max:
                            list_vehicle.append(make_model_engine + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make_model_engine + ' ' + str(year_min) + '-' + str(year_max))

                    dict_part[input_.loc[i, 'Url']]['vehicle'] = '\n'.join(sorted(list_vehicle))


                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Url': input_.loc[i, 'Url'],
                                             'Vehicle': dict_part[input_.loc[i, 'Url']]['vehicle']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Url': input_.loc[i, 'Url'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2955

[ok] - https://ecat.spectrapremium.com/en/parts/BMP09A
[尝试次数：1] - [剩余数量：2943] - [当前时间：16:21:52]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2942] - [当前时间：16:21:53]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP02A
[尝试次数：1] - [剩余数量：2941] - [当前时间：16:21:53]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2940] - [当前时间：16:21:53]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2939] - [当前时间：16:21:53]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP13A
[尝试次数：1] - [剩余数量：2938] - [当前时间：16:21:54]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP10A
[尝试次数：1] - [剩余数量：2937] - [当前时间：16:21:54]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP08A
[尝试次数：1] - [剩余数量：2936] - [当前时间：16:21:54]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP13A
[尝试次数：1] - [剩余数量：2935] - [当前时间：16:21:54]

[ok] - https://ecat.spectrapremium.com/en/parts/BMP06A
[尝试次数：1] - [剩余数量：2934] - [当前时间：16:21:54]

[ok] - https://ecat.

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：1] - [剩余数量：2858] - [当前时间：16:23:18]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：1] - [剩余数量：2857] - [当前时间：16:23:18]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：1] - [剩余数量：2856] - [当前时间：16:23:19]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：1] - [剩余数量：2855] - [当前时间：16:23:20]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：1] - [剩余数量：2854] - [当前时间：16:23:20]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：1] - [剩余数量：2853] - [当前时间：16:23:20]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：1] - [剩余数量：2852] - [当前时间：16:23:21]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：1] - [剩余数量：2851] - [当前时间：16:23:21]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP01A
[尝试次数：1] - [剩余数量：2850] - [当前时间：16:23:21]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP02A
[尝试次数：1] - [剩余数量：2849] - [当前时间：16:23:21]

[ok] - https://ecat.spectrapre

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2773] - [当前时间：16:23:48]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2772] - [当前时间：16:23:49]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2771] - [当前时间：16:23:49]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2770] - [当前时间：16:23:49]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP03A
[尝试次数：2] - [剩余数量：2769] - [当前时间：16:23:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2768] - [当前时间：16:23:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2767] - [当前时间：16:23:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2766] - [当前时间：16:23:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2765] - [当前时间：16:23:50]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP04A
[尝试次数：1] - [剩余数量：2764] - [当前时间：16:23:50]

[ok] - https://ecat.spectrapre

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2687] - [当前时间：16:24:15]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2686] - [当前时间：16:24:15]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2685] - [当前时间：16:24:16]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2684] - [当前时间：16:24:16]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP06A
[尝试次数：1] - [剩余数量：2683] - [当前时间：16:24:17]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：1] - [剩余数量：2682] - [当前时间：16:24:17]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：1] - [剩余数量：2681] - [当前时间：16:24:18]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：1] - [剩余数量：2680] - [当前时间：16:24:18]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：2] - [剩余数量：2679] - [当前时间：16:24:18]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP08A
[尝试次数：2] - [剩余数量：2678] - [当前时间：16:24:18]

[ok] - https://ecat.spectrapre

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2602] - [当前时间：16:24:53]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2601] - [当前时间：16:24:54]

[ok] - https://ecat.spectrapremium.com/en/parts/CMP03A
[尝试次数：3] - [剩余数量：2600] - [当前时间：16:24:55]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：5] - [剩余数量：2599] - [当前时间：16:24:56]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：6] - [剩余数量：2598] - [当前时间：16:24:56]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：2] - [剩余数量：2597] - [当前时间：16:24:57]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2596] - [当前时间：16:24:57]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2595] - [当前时间：16:24:58]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2594] - [当前时间：16:24:58]

[ok] - https://ecat.spectrapremium.com/en/parts/CRP05A
[尝试次数：1] - [剩余数量：2593] - [当前时间：16:24:58]

[ok] - https://ecat.spectrapre

...

[ok] - https://ecat.spectrapremium.com/en/parts/SZP02A
[尝试次数：1] - [剩余数量：371] - [当前时间：16:41:31]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP01A
[尝试次数：1] - [剩余数量：370] - [当前时间：16:41:31]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP03B
[尝试次数：1] - [剩余数量：369] - [当前时间：16:41:31]

[ok] - https://ecat.spectrapremium.com/en/parts/SZP04A
[尝试次数：1] - [剩余数量：368] - [当前时间：16:41:32]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP02A
[尝试次数：1] - [剩余数量：367] - [当前时间：16:41:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP02A
[尝试次数：1] - [剩余数量：366] - [当前时间：16:41:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP01A
[尝试次数：1] - [剩余数量：365] - [当前时间：16:41:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP02B
[尝试次数：1] - [剩余数量：364] - [当前时间：16:41:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP02B
[尝试次数：1] - [剩余数量：363] - [当前时间：16:41:33]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP03A
[尝试次数：1] - [剩余数量：362] - [当前时间：16:41:34]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/TOP34B
[尝试次数：1] - [剩余数量：285] - [当前时间：16:42:18]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP34B
[尝试次数：1] - [剩余数量：284] - [当前时间：16:42:20]

[ok] - https://ecat.spectrapremium.com/en/parts/OPA001
[尝试次数：2] - [剩余数量：283] - [当前时间：16:42:22]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP34B
[尝试次数：1] - [剩余数量：282] - [当前时间：16:42:23]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP35A
[尝试次数：1] - [剩余数量：281] - [当前时间：16:42:24]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP32A
[尝试次数：1] - [剩余数量：280] - [当前时间：16:42:30]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP35A
[尝试次数：1] - [剩余数量：279] - [当前时间：16:42:31]

[ok] - https://ecat.spectrapremium.com/en/parts/SUP04B
[尝试次数：2] - [剩余数量：278] - [当前时间：16:42:32]

[ok] - https://ecat.spectrapremium.com/en/parts/TOP35A
[尝试次数：1] - [剩余数量：277] - [当前时间：16:42:33]

[ok] - https://ecat.spectrapremium.com/en/parts/SUP06A
[尝试次数：2] - [剩余数量：276] - [当前时间：16:42:33]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：198] - [当前时间：16:43:09]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：197] - [当前时间：16:43:09]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：196] - [当前时间：16:43:11]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：195] - [当前时间：16:43:11]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：194] - [当前时间：16:43:12]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：193] - [当前时间：16:43:13]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：192] - [当前时间：16:43:14]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：191] - [当前时间：16:43:14]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP09A
[尝试次数：1] - [剩余数量：190] - [当前时间：16:43:15]

[ok] - https://ecat.spectrapremium.com/en/parts/VOP10A
[尝试次数：1] - [剩余数量：189] - [当前时间：16:43:15]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：111] - [当前时间：16:43:49]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：1] - [剩余数量：110] - [当前时间：16:43:50]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP34A
[尝试次数：1] - [剩余数量：109] - [当前时间：16:43:50]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：108] - [当前时间：16:43:51]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：107] - [当前时间：16:43:51]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：106] - [当前时间：16:43:52]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：105] - [当前时间：16:43:52]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：104] - [当前时间：16:43:52]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：103] - [当前时间：16:43:53]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：1] - [剩余数量：102] - [当前时间：16:43:53]

[ok] - https://ecat.spectrapremium.com/e

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：1] - [剩余数量：23] - [当前时间：16:44:20]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP56A
[尝试次数：1] - [剩余数量：22] - [当前时间：16:44:21]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP57A
[尝试次数：1] - [剩余数量：21] - [当前时间：16:44:21]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP59A
[尝试次数：1] - [剩余数量：20] - [当前时间：16:44:22]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP59A
[尝试次数：1] - [剩余数量：19] - [当前时间：16:44:23]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP57A
[尝试次数：1] - [剩余数量：18] - [当前时间：16:44:23]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP59A
[尝试次数：1] - [剩余数量：17] - [当前时间：16:44:24]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP59A
[尝试次数：1] - [剩余数量：16] - [当前时间：16:44:24]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP35A
[尝试次数：2] - [剩余数量：15] - [当前时间：16:44:24]

[ok] - https://ecat.spectrapremium.com/en/parts/VWP57A
[尝试次数：1] - [剩余数量：14] - [当前时间：16:44:24]

[ok] - https://ecat.spectrapremium.com/en/parts/VW